In [1]:
%matplotlib inline
from datetime import datetime
from pynwb import NWBFile
import numpy as np
from scipy import signal 
import pandas as pd
from pynwb.ecephys import ElectricalSeries

/Users/alexgonzalez/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
datHead = NWBFile('Tetrode Test', 'Data formating test using NWB', 'Li_T3g_060418', datetime(2018,6,4),
                  experimenter='Alex Gonzalez',
                  lab='Giocomo Lab',
                  institution='Stanford University',
                  experiment_description='Recording during maze session, T3g.',
                  session_id='20')
device = datHead.create_device(name='AlexRig',source='NeuralynxAck')

In [3]:
nTetrodes = 1
for tetrodeID in np.arange(nTetrodes):
    electrode_name = 'tetrode%i' % tetrodeID
    source = ""
    description = "Platinum tetrode prepared by hand and DC electroplated."
    location = "MEC"

    electrode_group = datHead.create_electrode_group(electrode_name,
                                                     source=source,
                                                     description=description,
                                                     location=location,
                                                     device=device)

In [4]:
xPos = np.array([0.0,0,20,20])
yPos = np.array([0.0,20,0,20])
zPos = 0.0
for idx in np.arange(4):
    datHead.add_electrode(idx,
                          x=xPos[idx], y=yPos[idx], z=zPos,
                          imp=float(250),
                          location='MEC', filtering='none',
                          description='channel %s' % idx, group=electrode_group)

In [5]:
electrode_table_region = datHead.create_electrode_table_region([0,1, 2,3], 'tetrode')

rate = 1000.0
np.random.seed(1234)
data_len = int(rate*60*5)
ephys_data = np.random.randn(data_len * 4)+np.random.poisson(0.1,data_len * 4)*2
ephys_data=ephys_data.reshape((data_len, 4))
np.save('ecephys_example',ephys_data)
ephys_timestamps = np.arange(data_len) / rate

ephys_ts = ElectricalSeries('test_ephys_data',
                            'an hypothetical source',
                            ephys_data,
                            electrode_table_region,
                            timestamps=ephys_timestamps,
                            # Alternatively, could specify starting_time and rate as follows
                            # starting_time=ephys_timestamps[0],
                            # rate=rate,
                            resolution=0.001,
                            comments="This data was randomly generated with numpy, using 1234 as the seed",
                            description="Random numbers generated with numpy.random.rand")
datHead.add_acquisition(ephys_ts)

In [6]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('ecephys_example.nwb', 'w') as io:
    io.write(datHead)

In [7]:
# import h5py
# with h5py.File('ecephys_example.nwb', 'r') as f:
#     for i in f['acquisition/test_ephys_data/data']:
#         print(i)
# # io = NWBHDF5IO('ecephys_example.nwb', 'r')
# # nwbfile = io.read()

In [18]:
ephys_data = np.random.randn(data_len * 4)+np.random.poisson(0.1,data_len * 4)*2
ephys_data=ephys_data.reshape((data_len, 4))
np.save('ecephys_example',ephys_data)


In [30]:
dat_slice=ephys_data[0:2][0:4]
#sarray.astype('int16').tofile(filename)


array([[ 0.47143516, -1.19097569,  1.43270697, -0.3126519 ],
       [-0.72058873,  0.88716294,  0.85958841, -0.6365235 ]])